# Hierarchical Regression in Python

This is a walkthrough of how to do hiearchial models in python, based on the blog post from the university of Virgina (https://data.library.virginia.edu/hierarchical-linear-regression/). This is **not** a notebook about the bayseian modelling technique. I was getting annoyed that nothing was done in Python!!  


## What is Hierarchical Regression Modelling?

Hierarchical Regression Modelling is a technique used to examine if a covariate of interest is a statistically significant explainer of variance in the dependent variable, after accounting for other variables. This can tell us if a covariate is an important predictor of the dependent variable. 

### Ok so how do I do it?

**Before we continue** covariates must ***only*** be added to models if there is sufficient a priori reason to include them, no p-hacking!! 

Hierarchical Regression Modelling is done by building models of more complexity by adding in more covariates. What we are interested is does a covariate of interest statistically signficiantly explain variation in the dependent variable.

So conceptually we:

1. Build sequential regression models adding more covariates
2. Run ANOVAs to get the total $SS$ (sum of squares)
3. Compare the total $SS$ between models and find the corresponding F statistic and P values

## Example in python.

In this example we will examine the relationship between happiness and social interaction. We know that number of friends in addition to age and gender is a predictor of happiness. However, we are interested in if the number of pets is an import predictor of happiness. 

Therefore our hypotheisis is that pets are an important predictor of happiness. 

To test this out first we import the libaries needed. I have chosen to use statsmodels.formula.api ols as its syntax is the most similar to R programming language (*for people who are more familiar with R*)

In [4]:
import pandas as pd 
import statsmodels.api as sm
from statsmodels.formula.api import ols

Next we use pandas to import the dataset. It is also good practice to explore the dataset as well which we will do.

In [5]:
df = pd.read_csv('http://static.lib.virginia.edu/statlab/materials/data/hierarchicalRegressionData.csv')
df.shape

(100, 5)

In [6]:
df.columns

Index(['happiness', 'age', 'gender', 'friends', 'pets'], dtype='object')

In [7]:
df.dtypes

happiness     int64
age           int64
gender       object
friends       int64
pets          int64
dtype: object

In [8]:
df.head()

,happiness,age,gender,friends,pets
0,5,24,Male,12,3
1,5,28,Male,8,1
2,6,25,Female,6,0
3,4,26,Male,4,2
4,3,20,Female,8,0


We should also check if there are null values in the dataframe

In [9]:
df.isnull().values.any()

False

After examining the data we can build our linear regression models. The first model will just examine happiness to get the Total SS, we will then build a model with age and gender, then a third one with friends and finally pets.

In [10]:
m0=ols('happiness ~ 1', data=df).fit() 

#add in age and gender
m1=ols('happiness ~ age + gender', data=df).fit() 

#add in friends
m2=ols('happiness ~ age + gender + friends', data=df).fit() 

#Now added in pets which we hypotheses is an important predictor 
m3=ols('happiness ~ age + gender + friends + pets', data=df).fit()

After building our models we run anovas to compute the total Sum of Sqaures

In [11]:
res_m0=sm.stats.anova_lm(m0)
res_m0

,df,sum_sq,mean_sq,F,PR(>F)
Residual,99.0,240.84,2.432727,NaN,NaN


In [12]:
res_m1=sm.stats.anova_lm(m1)
res_m1

,df,sum_sq,mean_sq,F,PR(>F)
gender,1.0,0.363329,0.363329,0.150633,0.698781
age,1.0,6.511470,6.511470,2.699601,0.103611
Residual,97.0,233.965200,2.412012,NaN,NaN


In [13]:
res_m2=sm.stats.anova_lm(m2)
res_m2

,df,sum_sq,mean_sq,F,PR(>F)
gender,1.0,0.363329,0.363329,0.166673,0.683995
age,1.0,6.511470,6.511470,2.987062,0.087147
friends,1.0,24.695651,24.695651,11.328846,0.001099
Residual,96.0,209.269549,2.179891,NaN,NaN


In [14]:
res_m3=sm.stats.anova_lm(m3)
res_m3

,df,sum_sq,mean_sq,F,PR(>F)
gender,1.0,0.363329,0.363329,0.178449,0.673664
age,1.0,6.511470,6.511470,3.198112,0.076911
friends,1.0,24.695651,24.695651,12.129281,0.000752
pets,1.0,15.846149,15.846149,7.782844,0.006374
Residual,95.0,193.423400,2.036036,NaN,NaN


As you can see as we add more variables the total sum squares decreases. We can also see that pets accounts for an additional 15.85 of the $SS$ and is a statistical significantly (*p=0.006374*) explainer of variation in happiness

**REMEMBER** that the $R^2$ will always increase when more variables are added. Thats why we need an f statistic and P value rather than relying on the increase in $R^2$ 

Lets take a look at the model with pets in it

In [15]:
m3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              happiness   R-squared:                       0.197
Model:                            OLS   Adj. R-squared:                  0.163
Method:                 Least Squares   F-statistic:                     5.822
Date:                Thu, 17 Dec 2020   Prob (F-statistic):           0.000311
Time:                        11:47:35   Log-Likelihood:                -174.88
No. Observations:                 100   AIC:                             359.8
Df Residuals:                      95   BIC:                             372.8
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          5.7854      1.903      3.041      0.003       2.008       9.563
gender[T.Male]    -0.1427      0.312     -0.458      0.648      -0.761       0.476
age               -0.1115      0.073     -1.525      0.131      -0.257       0.034
friends            0.1713      0.055      3.120      0.002       0.062       0.280
pets               0.3639      0.130      2.790      0.006       0.105       0.623
==============================================================================
Omnibus:                        0.717   Durbin-Watson:                   1.848
Prob(Omnibus):                  0.699   Jarque-Bera (JB):                0.832
Skew:                           0.122   Prob(JB):                        0.660
Kurtosis:                       2.625   Cond. No.                         353.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

We can see from this regression model that pets have a positive association (I am from a psychology background!) to happiness, that when someone has (0.36) pets there happiness increases by one point.


We can also work out effect size for the pets now by using Cohen's $F^2$

$F^2 = (R^2 included - R^2 excluded) \div (1-R^2 included)$

So first lets build a function to calculate the $F^2$:

In [16]:
def cohensf(r2_included,r2_excluded):
    f= (r2_included-r2_excluded)/(1-r2_included)
    return f

Now lets get the r2 from the two models with pets and without pets and assign them to a variable

In [27]:
r2_included=m3.rsquared
r2_excluded=m2.rsquared

print(f'R2 with pets = {r2_included} \nR2 without pets = {r2_excluded}')

R2 with pets = 0.19688008516445088 
R2 without pets = 0.13108474715995277


Finally lets get our Cohen's $F^2$:

In [31]:
f2= cohensf(r2_included,r2_excluded)
print(f'f2 is {f2}')

f2 is 0.0819246749944816


This is a tiny effect size and shows that pets are a statistically significant explainer of the variation in happiness, however the amount of variation it explains isn't great.